#Building a Recommender System with the MovieLens Data Set
> I wanted to build a recommender system using the MovieLens data set trying out both user-based collaborative filtering and item-based collaborative filtering and exploring the pros and cons of both.

##The Data
I used the MovieLens 100k data set. This is downloadable from their website and consists of 100,000 ratings from 943 users on 1,682 movies. The data is pre-cleaned with users having less than 20 ratings being removed.

##Exploration

In [442]:
import pandas as pd
import numpy as np

# pass in column names for each CSV
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('data/ml-100k/u.user', sep='|', names=u_cols)

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('data/ml-100k/u.data', sep='\t', names=r_cols, encoding='ISO-8859-1')

# the movies file contains columns indicating the movie's genres
# let's only load the first five columns of the file with usecols
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('data/ml-100k/u.item', sep='|', names=m_cols, usecols=range(5), encoding='ISO-8859-1')

# create one merged DataFrame
movie_ratings = pd.merge(movies, ratings)
lens = pd.merge(movie_ratings, users)

lens.head()

,movie_id,title,release_date,video_release_date,imdb_url,user_id,rating,unix_timestamp,age,sex,occupation,zip_code
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,308,4,887736532,60,M,retired,95076
1,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,308,5,887737890,60,M,retired,95076
2,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),308,4,887739608,60,M,retired,95076
3,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,308,4,887738847,60,M,retired,95076
4,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),308,5,887736696,60,M,retired,95076


###Top 10 most rated movies

In [430]:
# gets counts for each movie title and order results
most_rated = lens.groupby('title').size().order(ascending=False)[:10]
print(most_rated)

title
Star Wars (1977)                 583
Contact (1997)                   509
Fargo (1996)                     508
Return of the Jedi (1983)        507
Liar Liar (1997)                 485
English Patient, The (1996)      481
Scream (1996)                    478
Toy Story (1995)                 452
Air Force One (1997)             431
Independence Day (ID4) (1996)    429
dtype: int64


###Top 10 highest rated movies

In [431]:
# get average score for each movie
movie_stats = lens.groupby('title').agg({'rating': [np.size, np.mean]})

# only report movies with at least 100 ratings
atleast_100 = movie_stats['rating']['size'] >= 100
print(movie_stats[atleast_100].sort([('rating', 'mean')], ascending=False)[:15])

                                       rating          
                                         size      mean
title                                                  
Close Shave, A (1995)                     112  4.491071
Schindler's List (1993)                   298  4.466443
Wrong Trousers, The (1993)                118  4.466102
Casablanca (1942)                         243  4.456790
Shawshank Redemption, The (1994)          283  4.445230
Rear Window (1954)                        209  4.387560
Usual Suspects, The (1995)                267  4.385768
Star Wars (1977)                          583  4.358491
12 Angry Men (1957)                       125  4.344000
Citizen Kane (1941)                       198  4.292929
To Kill a Mockingbird (1962)              219  4.292237
One Flew Over the Cuckoo's Nest (1975)    264  4.291667
Silence of the Lambs, The (1991)          390  4.289744
North by Northwest (1959)                 179  4.284916
Godfather, The (1972)                     413  4

###A note on the sparsity of the data set
With 943 users and 1,682 movies we have a total of 1,586,126 possible ratings. We only have 100,000 ratings, meaning we are missing **93.70%** of all the possible data!

##Feature Selection
There is demographic information for the users as well as genre information for each movie. I wanted to start simple so I ended up only using the ratings data for each movie.

##User-Based Collaborative Filtering

In [432]:
# import recommendations library
import recommendations
from importlib import reload
reload(recommendations);

In [433]:
# build a dictionary of preferences by user
prefs = recommendations.loadMovieLens()

In [441]:
# let's look at the preferences for a particular user
prefs['37']

{'Alien (1979)': 4.0,
 'Alien 3 (1992)': 3.0,
 'Aliens (1986)': 4.0,
 'Arrival, The (1996)': 2.0,
 'Bad Boys (1995)': 4.0,
 'Batman (1989)': 5.0,
 'Batman Returns (1992)': 2.0,
 'Blade Runner (1982)': 4.0,
 'Booty Call (1997)': 4.0,
 'Braveheart (1995)': 5.0,
 'Broken Arrow (1996)': 3.0,
 'Bulletproof (1996)': 4.0,
 'Chain Reaction (1996)': 3.0,
 'Clear and Present Danger (1994)': 4.0,
 'Crow, The (1994)': 5.0,
 'Daylight (1996)': 3.0,
 'Demolition Man (1993)': 3.0,
 'Die Hard 2 (1990)': 5.0,
 'Die Hard: With a Vengeance (1995)': 4.0,
 'Dragonheart (1996)': 2.0,
 'Empire Strikes Back, The (1980)': 4.0,
 'Eraser (1996)': 5.0,
 'Escape from L.A. (1996)': 2.0,
 'Executive Decision (1996)': 3.0,
 'Fugitive, The (1993)': 4.0,
 'Glimmer Man, The (1996)': 3.0,
 'Godfather, The (1972)': 4.0,
 'Heat (1995)': 3.0,
 'Hunt for Red October, The (1990)': 4.0,
 'Independence Day (ID4) (1996)': 2.0,
 'Indiana Jones and the Last Crusade (1989)': 4.0,
 'Jurassic Park (1993)': 1.0,
 'Long Kiss Goodnight,

In [454]:
# goes through every other person in the prefs dictionary, calculates how similar they are, then multiplies each 
# item by the user's similarity score, adds up all the products, and takes the weighted average
"""
# Gets recommendations for a person by using a weighted average
# of every other user's rankings
def getRecommendationsByUser(prefs,person,similarity=sim_pearson):
  totals={}
  simSums={}
  for other in prefs:
    # don't compare me to myself
    if other==person: continue
    sim=similarity(prefs,person,other)

    # ignore scores of zero or lower
    if sim<=0: continue
    for item in prefs[other]:

      # only score movies I haven't seen yet
      if item not in prefs[person] or prefs[person][item]==0:
        # Similarity * Score
        totals.setdefault(item,0)
        totals[item]+=prefs[other][item]*sim
        # Sum of similarities
        simSums.setdefault(item,0)
        simSums[item]+=sim

  # Create the normalized list
  rankings=[(total/simSums[item],item) for item,total in totals.items()]

  # Return the sorted list rankings.sort()
  rankings.sort()
  rankings.reverse()
  return rankings
"""

recommendations.getRecommendationsByUser(prefs,'37')[0:30]

[(5.0, 'They Made Me a Criminal (1939)'),
 (5.0, "Someone Else's America (1995)"),
 (5.0, 'Santa with Muscles (1996)'),
 (5.0, 'Prefontaine (1997)'),
 (5.0, 'Marlene Dietrich: Shadow and Light (1996) '),
 (5.0, 'Little City (1998)'),
 (5.0, 'Great Day in Harlem, A (1994)'),
 (5.0, 'Boys, Les (1997)'),
 (5.0, 'Aiqing wansui (1994)'),
 (4.999999999999999, 'Saint of Fort Washington, The (1993)'),
 (4.801378113392982, 'Maya Lin: A Strong Clear Vision (1994)'),
 (4.671725730556882, 'Perfect Candidate, A (1996)'),
 (4.60714682819528, 'Legal Deceit (1997)'),
 (4.591670203439434, 'Angel Baby (1995)'),
 (4.576266675886999, 'Pather Panchali (1955)'),
 (4.563703394460864, 'Love in the Afternoon (1957)'),
 (4.561433337981203, 'Nénette et Boni (1996)'),
 (4.561376607987208, 'Anna (1996)'),
 (4.534775069641421, 'Paths of Glory (1957)'),
 (4.531030618477851, 'Everest (1998)'),
 (4.518684997945028, 'Quiet Room, The (1996)'),
 (4.507380405826082, 'Golden Earrings (1947)'),
 (4.495300269797751, 'Wallace

##Item-Based Collaborative Filtering

In [443]:
# first we need to build a dictionary of similar items
# this function inverts the prefs dictionary and calulates the top n most similar items for each item
itemsim=recommendations.calculateSimilarItems(prefs,n=50)

100 / 1664
200 / 1664
300 / 1664
400 / 1664
500 / 1664
600 / 1664
700 / 1664
800 / 1664
900 / 1664
1000 / 1664
1100 / 1664
1200 / 1664
1300 / 1664
1400 / 1664
1500 / 1664
1600 / 1664


In [437]:
itemsim[0:30]

{'Scream (1996)': [(1.0000000000000018, 'Infinity (1996)'),
  (1.0, 'Wedding Gift, The (1994)'),
  (1.0, 'Turbo: A Power Rangers Movie (1997)'),
  (1.0, 'Time Tracers (1995)'),
  (1.0, 'Sudden Manhattan (1996)'),
  (1.0, 'Scarlet Letter, The (1926)'),
  (1.0, 'Safe Passage (1994)'),
  (1.0, 'Rendezvous in Paris (Rendez-vous de Paris, Les) (1995)'),
  (1.0, 'Maya Lin: A Strong Clear Vision (1994)'),
  (1.0, 'Love and Death on Long Island (1997)'),
  (1.0, 'Line King: Al Hirschfeld, The (1996)'),
  (1.0, 'Calendar Girl (1993)'),
  (1.0, 'Buddy (1997)'),
  (1.0, 'Bewegte Mann, Der (1994)'),
  (1.0, '8 Seconds (1994)'),
  (0.999999999999998, 'Caro Diario (Dear Diary) (1994)'),
  (0.9961164901835046, 'Leading Man, The (1996)'),
  (0.9819805060619656, 'Zeus and Roxanne (1997)'),
  (0.9683296637314885, 'Bad Moon (1996)'),
  (0.9449111825230678, "Gilligan's Island: The Movie (1998)"),
  (0.9449111825230674, 'Frisk (1995)'),
  (0.8708635721768005, 'Full Speed (1996)'),
  (0.8685990362153793, 'A

In [448]:
# goes through all of a users items, gets similar items, weights them using their similarity score,
# adds up all the products, and takes the weighted average
"""
# Gets recommendations for a person by using a weighted list
# of items most similar to a user's top-rated items
def getRecommendationsByItems(prefs,itemMatch,user):
  userRatings=prefs[user]
  scores={}
  totalSim={}

  # Loop over items rated by this user
  for (item,rating) in userRatings.items():

    # Loop over items similar to this one
    for (similarity,item2) in itemMatch[item]:

      # Ignore if this user has already rated this item
      if item2 in userRatings: continue

      # Weighted sum of rating times similarity
      scores.setdefault(item2,0)
      scores[item2]+=similarity*rating

      # Sum of all the similarities
      totalSim.setdefault(item2,0)
      totalSim[item2]+=similarity

  # Divide each total score by total weighting to get an average
  rankings=[(score/totalSim[item],item) for item,score in scores.items()]

  # Return the rankings from highest to lowest
  rankings.sort()
  rankings.reverse()
  return rankings
"""

recommendations.getRecommendationsByItems(prefs,itemsim,'37')[0:30]

[(5.000000000000001, 'Air Bud (1997)'),
 (5.0, 'Wishmaster (1997)'),
 (5.0, 'Palookaville (1996)'),
 (5.0, 'Nelly & Monsieur Arnaud (1995)'),
 (5.0, 'Mercury Rising (1998)'),
 (5.0, 'Last Summer in the Hamptons (1995)'),
 (5.0, 'Kicking and Screaming (1995)'),
 (5.0, "I'm Not Rappaport (1996)"),
 (5.0, 'Good Man in Africa, A (1994)'),
 (5.0, 'French Twist (Gazon maudit) (1995)'),
 (5.0, 'Deceiver (1997)'),
 (5.0, 'Daytrippers, The (1996)'),
 (5.0, 'Country Life (1994)'),
 (5.0, 'Commandments (1997)'),
 (5.0, 'Charade (1963)'),
 (5.0, 'Career Girls (1997)'),
 (4.789750590601399, 'Zeus and Roxanne (1997)'),
 (4.742289906695968, 'Twisted (1996)'),
 (4.679955284098884, 'Last Dance (1996)'),
 (4.679214831609535, 'Curdled (1996)'),
 (4.67774081332102, 'When We Were Kings (1996)'),
 (4.666666666666667, 'Ruling Class, The (1972)'),
 (4.6519152645903565, 'Thieves (Voleurs, Les) (1996)'),
 (4.5931218011454895, 'Guilty as Sin (1993)'),
 (4.558481559887747, 'Above the Rim (1994)'),
 (4.52794221557

###Important note about speed
Although the item-similarity dictionary takes a while to create, once it's created, recommendations can be calculated a lot more quickly not having to run calculations over the entire data set like in the user-based method.

##Challenges and Possible Extensions

###Validation
Although I got each method working, I only validated the results empirically by grabbing recommendations for different users. I tried to do a train-test-split on my data set, but because my recommender system only returns top n recommendations, I was running into issues on not being able to make predictions on some of my test set and it would require a significant change to the code.
###Other techniques
I did not use any of the other features besides ratings. One thing I could have done was run clustering on either users or movies to subset the data, and then make predictions only within each subset. This would speed up calculations and possibly give better recommendations. Another technique I wanted to try was singular value decomposition. What this does is it decomposes users and movies by matrix factorization into a set of latent factors, like "horror" or "sci-fi". Recommendations are supposed to be very good and this was used to win the Netflix Prize.